In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract data from green_tripdata schema 0",
  "environments": ["production", "test"],
  "inputURI": "s3a://nyc-tlc/trip*data/green_tripdata_2013-08.csv",
  "outputView": "green_tripdata0_raw",            
  "delimiter": "Comma",
  "quote" : "DoubleQuote",
  "header": true,
  "persist": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply green_tripdata schema 0 data types",
  "environments": ["production", "test"],
  "schemaURI": "/home/jovyan/examples/tutorial/0/green_tripdata0.json",
  "inputView": "green_tripdata0_raw",            
  "outputView": "green_tripdata0"
}

In [ ]:
%sqlvalidate name="ensure no errors exist after data typing" environments=production,test
SELECT
  SUM(error) = 0 AS valid
  ,TO_JSON(
      NAMED_STRUCT(
        'count', COUNT(error), 
        'errors', SUM(error)
      )
  ) AS message
FROM (
  SELECT 
    CASE 
      WHEN SIZE(_errors) > 0 THEN 1 
      ELSE 0 
    END AS error 
  FROM green_tripdata0
) input_table

In [ ]:
%metadata truncate=1000
green_tripdata0

In [ ]:
{
  "type": "DeltaLakeLoad",
  "name": "write out green_tripdata0 dataset",
  "environments": ["production", "test"],
  "inputView": "green_tripdata0",
  "outputURI": "/home/jovyan/examples/tutorial/0/output/green_tripdata0.delta",
  "saveMode": "Overwrite"
}